# Notebook for computing Hypervolume

## Example for problem formulation: Nordhaus + Sufficientarian

In [1]:
import pandas as pd

archives = []
for i in range(1, 200):
    archive = pd.read_csv(f'./results/NORDHAUS/SUFFICIENTARIAN/archive_{i}.csv')
    archives.append(archive)

big_archive = pd.concat(archives)
big_archive


,Unnamed: 0,sr,miu,irstp,Total Aggregated Utility,Distance to threshold 2035,Distance to threshold 2055,Distance to threshold 2075,Population under threshold 2035,Population under threshold 2055,Population under threshold 2075
0,0,0.224019,2285.041456,0.001234,17213.162297,0.113453,0.0,0.0,265.114464,0.0,0.0
1,1,0.151142,2171.439718,0.001604,16130.187858,0.104972,0.0,0.0,265.114464,0.0,0.0
2,2,0.109935,2238.029382,0.002411,14039.136180,0.100266,0.0,0.0,265.114464,0.0,0.0
3,3,0.100983,2265.027269,0.003319,12140.526480,0.099245,0.0,0.0,265.114464,0.0,0.0
4,4,0.123296,2145.405225,0.002071,14890.561676,0.101738,0.0,0.0,265.114464,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2,2,0.133113,2065.146345,0.001000,17800.076455,0.102639,0.0,0.0,265.114464,0.0,0.0
3,3,0.100007,2065.009641,0.001000,17672.929081,0.098822,0.0,0.0,265.114464,0.0,0.0
4,4,0.105873,2065.077081,0.001000,17700.072498,0.099499,0.0,0.0,265.114464,0.0,0.0
5,5,0.151902,2065.064505,0.001000,17850.027736,0.104805,0.0,0.0,265.114464,0.0,0.0


## Collection of solutions for each generation

In [2]:
from platypus import Solution, Problem

n_rbfs = 4  # What's this?
n_objs = 7
n_vars = n_rbfs * 8

problem = Problem(n_vars, n_objs)

# matters for hypervolume
problem.directions[0] = Problem.MAXIMIZE  # Total aggregated utility
problem.directions[1] = Problem.MINIMIZE  # Distance to threshold 2035
problem.directions[2] = Problem.MINIMIZE  # Distance to threshold 2055
problem.directions[3] = Problem.MINIMIZE  # Distance to threshold 2075
problem.directions[4] = Problem.MAXIMIZE  # Population under threshold 2035
problem.directions[5] = Problem.MINIMIZE  # Population under threshold 2055
problem.directions[6] = Problem.MINIMIZE  # Population under threshold 2075

In [3]:
archives = {}
for nfe, generation in big_archive.groupby("Unnamed: 0"):
    # we slice from 2, getting rid of the first two columns
    # which contain the NFE and ID of each solution
    archive = []
    for i, row in generation.iloc[:, 1::].iterrows():
        solution = Solution(problem)
        solution.objectives = row
        archive.append(solution)
    archives[nfe] = archive


## Reference Set

In [4]:
data = pd.read_csv('./results/NORDHAUS/SUFFICIENTARIAN/archive_199.csv')

ref_set = []
for i, row in data.iterrows():
    solution = Solution(problem)
    solution.objectives = row
    ref_set.append(solution)

## Compute Hypervolume

In [5]:
from platypus import Hypervolume

hv = Hypervolume(reference_set=ref_set)

nfes = []
hvs = []
for nfe, archive in archives.items():
    nfes.append(nfe)
    hvs.append(hv.calculate(archive))


PlatypusError: objective with empty range

## Plotting

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("white")

fig, ax = plt.subplots()

ax.plot(nfes, hvs)
ax.set_xlabel('nfe')
ax.set_ylabel('hypervolume')

sns.despine()

plt.show()